<a href="https://colab.research.google.com/github/gabie0208/chinese-nlp/blob/main/1128_meaning_classification_xialai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 4.0 MB/s 
     |████████████████████████████████| 596 kB 65.2 MB/s 
     |████████████████████████████████| 895 kB 67.4 MB/s 
     |████████████████████████████████| 59 kB 9.1 MB/s 
     |████████████████████████████████| 3.3 MB 63.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch
import numpy as np
import transformers
from tqdm import tqdm

from transformers import BertTokenizer, BertModel, AdamW
from transformers import BertForMaskedLM,BertForSequenceClassification

import torch.nn.functional as F
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import random
import time
import datetime

In [3]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [35]:
def read_file(filename):
    with open(filename, "r") as file:
        lines = file.readlines()
        l = []
        for line in lines:
            l.append(line.strip().split("\n"))
    return l

buyu_xialai_test = read_file("/content/xialai-test data.txt")
buyu_xialai_train = read_file("/content/xialai-train data.txt")

In [36]:
buyu_xialai_train[:10]

[['接下来是马和羊。\t4'],
 ['走在大街上，我就注意看广告或者标语；遇见不明白的地方，我就记下来，以后再问人或者查字典。\t2'],
 ['中国的大都会，我前半生住过的地方，不在少数；可是当一个人静下来回想起从前，上海的热闹，南京的辽阔，广州的乌烟瘴气，汉口武昌的杂乱无章，甚至于青岛的清幽，福州的秀丽，以及杭州的沉着，总归都比不上北平的典丽堂皇，悠闲清妙。\t3'],
 ['从以上看出，中国的节日可分为四类：第一类是传统节日，是几千年、几百年延续下来的，来历都有民间传说或历史故事，一般与鬼、神、天、地联系在一起。\t2'],
 ['这是旧时的艺人留下来的古话了。\t2'],
 ['我手中的笔不禁停了下来，怕把它惊跑。\t3'],
 ['与其被导游领着鱼贯而入、聆听讲解，最后雀跃而去，我宁愿在走累了的时候，找一个喝茶的地方坐下来，这才是极惬意的赏心乐事。\t1'],
 ['专家分析认为，不停忙碌的白领希望更多地与亲朋相处，这反映出人们想逃离现代生活的快节奏，让生活慢下来的期望。\t3'],
 ['学跳舞是一件很辛苦的事，每天都要对着镜子练习同样的动作，但在校长的鼓励下，她还是坚持了下来。\t2'],
 ['一切都按照应有的程序进行，把该炒的菜炒了，把酒打开，坐下来一边谈话一边吃东西，地方还是这个地方，只不过时间变了，人也不是那个人，这个时候周健的心情很复杂。\t1']]

In [37]:
def load_data(data_dir, *filenames):
    sentences = []

    for filename in filenames:
        with open(data_dir + filename, 'r') as f:
            for line in f.readlines():
                sentences.append(line.strip().split('\t')[0])
    return sentences, len(sentences)

In [38]:
def save_num(data_dir, *filenames):
  num = []
  for filename in filenames:
      with open(data_dir + filename, 'r') as f:
          for line in f.readlines():
              num.append(line.strip().split('\t')[1])
  return num

In [39]:
data_dir = '/content/'

xialai, total_xialai = load_data(data_dir, "xialai-train data.txt")
xialai_num = save_num(data_dir, "xialai-train data.txt")
print("下来 :", total_xialai)
print(len(xialai_num))

下来 : 5101
5101


In [40]:
MAX_LEN = 50

def preprocess(data, label):
    sentences = []
    labels = []
    nums = []
    error_cnt = 0
    for sent in data:
        # 해당 문장에 방향보어가 있는 지 확인
        if label not in sent:
            print(f"Sententce : {sent}")
            error_cnt += 1
            continue
        sent = sent[:MAX_LEN]
        #sent = sent.replace(label, '[MASK]')
        sentences.append(sent)
        labels.append(label)
    print(f"{label} 방향보어 없는 문장 개수 : {error_cnt}")
    return sentences, labels

In [41]:
xialai_sent, xialai_label= preprocess(xialai, '下来')

下来 방향보어 없는 문장 개수 : 0


In [42]:
xialai_sent[:5]

['接下来是马和羊。',
 '走在大街上，我就注意看广告或者标语；遇见不明白的地方，我就记下来，以后再问人或者查字典。',
 '中国的大都会，我前半生住过的地方，不在少数；可是当一个人静下来回想起从前，上海的热闹，南京的辽阔，广',
 '从以上看出，中国的节日可分为四类：第一类是传统节日，是几千年、几百年延续下来的，来历都有民间传说或历',
 '这是旧时的艺人留下来的古话了。']

In [43]:
xialai_num[:5]

['4', '2', '3', '2', '2']

In [44]:
train_sent = xialai_sent
train_num = xialai_num

In [45]:
MODEL_TYPE = 'bert-base-chinese'
MAX_SIZE = 100
BATCH_SIZE = 100

tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

In [46]:
tokenized_sent = [tokenizer.tokenize(sent) for sent in train_sent]
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_sent]

In [47]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 100
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([2970,  678, 3341, 3221, 7716, 1469, 5399,  511,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])

In [48]:
num_list = ["1", "2", "3", "4"]
nums = [num_list.index(x) for x in train_num]

In [49]:
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [50]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_nums, validation_nums = train_test_split(input_ids, nums, random_state=1, test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=1, test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
#train_labels = torch.tensor(train_labels)
train_nums = torch.tensor(train_nums)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
#validation_labels = torch.tensor(validation_labels)
validation_nums = torch.tensor(validation_nums)
validation_masks = torch.tensor(validation_masks)

In [51]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_nums)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_nums)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [52]:
model = BertForSequenceClassification.from_pretrained(MODEL_TYPE, num_labels=4)
model.cuda()

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [53]:
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW, BertConfig

# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [54]:
# 정확도 계산 함수
def flat_accuracy(preds, nums):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    nums_flat = nums.flatten()

    return np.sum(pred_flat == nums_flat) / len(nums_flat)

In [55]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [56]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

In [57]:
# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)

        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.73
  Training epcoh took: 0:00:45

Running Validation...
  Accuracy: 0.91
  Validation took: 0:00:02

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.25
  Training epcoh took: 0:00:45

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:02

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.15
  Training epcoh took: 0:00:45

Running Validation...
  Accuracy: 0.92
  Validation took: 0:00:02

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.08
  Training epcoh took: 0:00:45

Running Validation...
  Accuracy: 0.93
  Validation took: 0:00:02

Training complete!


In [58]:
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [59]:
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, nums = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_nums = nums.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_nums)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu()

    return logits

In [60]:
logits = test_sentences(['走在大街上，我就注意看广告或者标语；遇见不明白的地方，我就记下来，以后再问人或者查字典。']) #answer 2
num_list = ["1", "2", "3", "4"]

print(logits)
print(num_list[np.argmax(logits)])

tensor([[-1.2274,  4.9765, -1.2487, -1.9121]])
2


In [61]:
logits = test_sentences(['一切都按照应有的程序进行，把该炒的菜炒了，把酒打开，坐下来一边谈话一边吃东西，地方还是这个地方，只不过时间变了，人也不是那个人，这个时候周健的心情很复杂。']) #answer 1
num_list = ["1", "2", "3", "4"]

print(logits)
print(num_list[np.argmax(logits)])

tensor([[ 4.4459, -2.5002, -1.7986, -2.1389]])
1


In [66]:
logits = test_sentences(['中国的大都会，我前半生住过的地方，不在少数；可是当一个人静下来回想起从前，上海的热闹，南京的辽阔，广州的乌烟瘴气，汉口武昌的杂乱无章，甚至于青岛的清幽，福州的秀丽，以及杭州的沉着，总归都比不上北平的典丽堂皇，悠闲清妙。']) #answer 3
num_list = ["1", "2", "3", "4"]

print(logits)
print(num_list[np.argmax(logits)])

tensor([[-2.0106, -0.7118,  5.3210, -1.4339]])
3


In [62]:
import torch.nn.functional as F

In [63]:
def predict_sentence():
    sent = input('Input Sentence')
    logits = test_sentences([sent])
    softmax = F.softmax(logits[0], dim=0).numpy()
    
    for l, p in zip(num_list, softmax):
        print(f"{l} : {p*100:.2f}%")

In [64]:
predict_sentence()

Input Sentence走在大街上，我就注意看广告或者标语；遇见不明白的地方，我就记下来，以后再问人或者查字典。
1 : 0.20%
2 : 99.50%
3 : 0.20%
4 : 0.10%


In [65]:
predict_sentence()

Input Sentence一切都按照应有的程序进行，把该炒的菜炒了，把酒打开，坐下来一边谈话一边吃东西，地方还是这个地方，只不过时间变了，人也不是那个人，这个时候周健的心情很复杂。
1 : 99.57%
2 : 0.10%
3 : 0.19%
4 : 0.14%


In [67]:
predict_sentence()

Input Sentence中国的大都会，我前半生住过的地方，不在少数；可是当一个人静下来回想起从前，上海的热闹，南京的辽阔，广州的乌烟瘴气，汉口武昌的杂乱无章，甚至于青岛的清幽，福州的秀丽，以及杭州的沉着，总归都比不上北平的典丽堂皇，悠闲清妙。
1 : 0.07%
2 : 0.24%
3 : 99.58%
4 : 0.12%
